In [1]:
!pip install cirq numpy scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [1]:
import cirq
import numpy as np
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Rastgelelik kontrolü için sabit bir tohum
np.random.seed(42)

# Kuantum Özellik Haritası Tanımlama
class QuantumFeatureMap:
    def __init__(self, num_qubits: int):
        self.num_qubits = num_qubits
        self.circuit = self._build_circuit()

    def _build_circuit(self):
        qubits = [cirq.GridQubit(i, 0) for i in range(self.num_qubits)]
        circuit = cirq.Circuit()
        for qubit in qubits:
            circuit.append(cirq.H(qubit))  # Hadamard kapıları
        if self.num_qubits > 1:
            for i in range(self.num_qubits - 1):
                circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))  # CNOT kapıları
        return circuit

    def get_circuit(self):
        return self.circuit

# Kuantum Özellik Dönüşümü
def quantum_feature_transformation(data, num_qubits):
    feature_map = QuantumFeatureMap(num_qubits)
    # Deterministik matris
    transformation_matrix = np.ones((data.shape[1], data.shape[1]))  # Sabit bir matris
    transformed_data = np.dot(data, transformation_matrix)
    return transformed_data

# Veri Setini Yükleme ve Hazırlama
data, target = load_wine(return_X_y=True)

# Veriyi Ölçeklendirme ve Pozitif Hale Getirme
scaler = StandardScaler()
data = scaler.fit_transform(data)

# Veriyi Pozitif Hale Getirme
minmax_scaler = MinMaxScaler()
data = minmax_scaler.fit_transform(data)

# Eğitim ve Test Setlerine Ayırma
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# Özelliklerin Performansını Değerlendirme
num_runs = 5  # Kodun birkaç kez çalıştırılması
all_feature_accuracies = np.zeros((num_runs, data.shape[1]))

for run in range(num_runs):
    feature_accuracies = []
    for i in range(data.shape[1]):
        X_train_transformed = quantum_feature_transformation(X_train[:, i:i+1], num_qubits=1)
        X_test_transformed = quantum_feature_transformation(X_test[:, i:i+1], num_qubits=1)

        # Destek Vektör Makinesi (SVM) ile Sınıflandırma
        clf = SVC(kernel='linear')
        clf.fit(X_train_transformed, y_train)
        y_pred = clf.predict(X_test_transformed)

        # Test Doğruluğunu Hesaplama
        accuracy = accuracy_score(y_test, y_pred)
        feature_accuracies.append(accuracy)

    all_feature_accuracies[run, :] = feature_accuracies

# Ortalama Doğrulukları Hesaplama
average_accuracies = np.mean(all_feature_accuracies, axis=0)

# En İyi Özelliklerin Seçilmesi
num_best_features = 5  # İstediğiniz en iyi öznitelik sayısını ayarlayın
best_feature_indices = np.argsort(average_accuracies)[-num_best_features:]
print(f"En İyi {num_best_features} Özelliklerin İndeksleri:", best_feature_indices)

# En yüksek doğruluk oranına sahip öznitelikleri yazdırma
for i in best_feature_indices:
    print(f"Öznitelik {i} en yüksek doğruluk oranına sahip: Doğruluk = {average_accuracies[i]}")


En İyi 5 Özelliklerin İndeksleri: [ 5  9  0  6 12]
Öznitelik 5 en yüksek doğruluk oranına sahip: Doğruluk = 0.75
Öznitelik 9 en yüksek doğruluk oranına sahip: Doğruluk = 0.75
Öznitelik 0 en yüksek doğruluk oranına sahip: Doğruluk = 0.7777777777777778
Öznitelik 6 en yüksek doğruluk oranına sahip: Doğruluk = 0.7777777777777778
Öznitelik 12 en yüksek doğruluk oranına sahip: Doğruluk = 0.7777777777777778


In [2]:
import cirq
import numpy as np
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Rastgelelik kontrolü için sabit bir tohum
np.random.seed(42)

# Kuantum Özellik Haritası Tanımlama
class QuantumFeatureMap:
    def __init__(self, num_qubits: int):
        self.num_qubits = num_qubits
        self.circuit = self._build_circuit()

    def _build_circuit(self):
        qubits = [cirq.GridQubit(i, 0) for i in range(self.num_qubits)]
        circuit = cirq.Circuit()
        for qubit in qubits:
            circuit.append(cirq.H(qubit))  # Hadamard kapıları
        if self.num_qubits > 1:
            for i in range(self.num_qubits - 1):
                circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))  # CNOT kapıları
        return circuit

    def get_circuit(self):
        return self.circuit

# Kuantum Özellik Dönüşümü
def quantum_feature_transformation(data, num_qubits):
    feature_map = QuantumFeatureMap(num_qubits)
    # Deterministik matris
    transformation_matrix = np.ones((data.shape[1], data.shape[1]))  # Sabit bir matris
    transformed_data = np.dot(data, transformation_matrix)
    return transformed_data

# Veri Setini Yükleme ve Hazırlama
data, target = load_wine(return_X_y=True)

# Veriyi Ölçeklendirme ve Pozitif Hale Getirme
scaler = StandardScaler()
data = scaler.fit_transform(data)

# Veriyi Pozitif Hale Getirme
minmax_scaler = MinMaxScaler()
data = minmax_scaler.fit_transform(data)

# Eğitim ve Test Setlerine Ayırma
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# Özelliklerin Performansını Değerlendirme
num_runs = 5  # Kodun birkaç kez çalıştırılması
all_feature_accuracies = np.zeros((num_runs, data.shape[1]))

for run in range(num_runs):
    feature_accuracies = []
    for i in range(data.shape[1]):
        X_train_transformed = quantum_feature_transformation(X_train[:, i:i+1], num_qubits=1)
        X_test_transformed = quantum_feature_transformation(X_test[:, i:i+1], num_qubits=1)

        # Destek Vektör Makinesi (SVM) ile Sınıflandırma
        clf = SVC(kernel='linear')
        clf.fit(X_train_transformed, y_train)
        y_pred = clf.predict(X_test_transformed)

        # Test Doğruluğunu Hesaplama
        accuracy = accuracy_score(y_test, y_pred)
        feature_accuracies.append(accuracy)

    all_feature_accuracies[run, :] = feature_accuracies

# Ortalama Doğrulukları Hesaplama
average_accuracies = np.mean(all_feature_accuracies, axis=0)

# En İyi Özelliklerin Seçilmesi
num_best_features = 5  # İstediğiniz en iyi öznitelik sayısını ayarlayın
best_feature_indices = np.argsort(average_accuracies)[-num_best_features:]
print(f"En İyi {num_best_features} Özelliklerin İndeksleri:", best_feature_indices)

# En iyi öznitelikler ile veri setinin oluşturulması
X_train_best = X_train[:, best_feature_indices]
X_test_best = X_test[:, best_feature_indices]

# Modeli Tüm Özelliklerle Eğitme
clf_all_features = SVC(kernel='linear')
clf_all_features.fit(X_train, y_train)
y_pred_all_features = clf_all_features.predict(X_test)
accuracy_all_features = accuracy_score(y_test, y_pred_all_features)

# Modeli En İyi Özelliklerle Eğitme
clf_best_features = SVC(kernel='linear')
clf_best_features.fit(X_train_best, y_train)
y_pred_best_features = clf_best_features.predict(X_test_best)
accuracy_best_features = accuracy_score(y_test, y_pred_best_features)

# Sonuçları Yazdırma
print(f"Tüm Özellikler ile Doğruluk: {accuracy_all_features}")
print(f"En İyi Özellikler ile Doğruluk: {accuracy_best_features}")

# En yüksek doğruluk oranına sahip öznitelikleri yazdırma
for i in best_feature_indices:
    print(f"Öznitelik {i} en yüksek doğruluk oranına sahip: Doğruluk = {average_accuracies[i]}")


En İyi 5 Özelliklerin İndeksleri: [ 5  9  0  6 12]
Tüm Özellikler ile Doğruluk: 1.0
En İyi Özellikler ile Doğruluk: 1.0
Öznitelik 5 en yüksek doğruluk oranına sahip: Doğruluk = 0.75
Öznitelik 9 en yüksek doğruluk oranına sahip: Doğruluk = 0.75
Öznitelik 0 en yüksek doğruluk oranına sahip: Doğruluk = 0.7777777777777778
Öznitelik 6 en yüksek doğruluk oranına sahip: Doğruluk = 0.7777777777777778
Öznitelik 12 en yüksek doğruluk oranına sahip: Doğruluk = 0.7777777777777778


In [3]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Rastgelelik kontrolü için sabit bir tohum
np.random.seed(42)

# Veri Setini Yükleme ve Hazırlama
data, target = load_wine(return_X_y=True)

# Veriyi Ölçeklendirme
scaler = StandardScaler()
data = scaler.fit_transform(data)

# Eğitim ve Test Setlerine Ayırma
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# Modellerin Tanımlanması
models = {
    "SVM": SVC(kernel='linear'),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000)
}

# Modelleri Eğitim ve Test Etme
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy

# Sonuçları Yazdırma
print("Klasik Makine Öğrenmesi Modellerinin Sonuçları:")
for model_name, accuracy in results.items():
    print(f"{model_name}: Doğruluk = {accuracy:.4f}")


Klasik Makine Öğrenmesi Modellerinin Sonuçları:
SVM: Doğruluk = 0.9722
Decision Tree: Doğruluk = 0.9444
Random Forest: Doğruluk = 1.0000
Logistic Regression: Doğruluk = 1.0000
